In [ ]:
%load_ext watermark


In [ ]:
import itertools as it

from IPython.display import display
import pandas as pd
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp

from pylib.munge._calc_fixprobs_from_traits import calc_fixprobs_from_traits
from pylib.viz._get_nben_cmap import get_nben_cmap
from pylib.viz._get_nben_hue_norm import get_nben_hue_norm
from pylib.viz._size_fixation_areaplot import size_fixation_areaplot
from pylib.viz._size_fixation_cliffplot import size_fixation_cliffplot
from pylib.viz._size_fixation_heatmap import size_fixation_heatmap


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "cupy-traits"
teeplot_subdir


## Prep Data


In [ ]:
dfxs = pl.concat(
    [
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/s67d2/download"),
            ),
        ).with_columns(
            pl.lit("50/50").alias("initial conditions"),
            pl.lit("1D demes").alias("population structure"),
        ),
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/8q5v6/download"),
            ),
        ).with_columns(
            pl.lit("50/50").alias("initial conditions"),
            pl.lit("2D demes").alias("population structure"),
        ),
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/duam2/download"),
            ),
        ).with_columns(
            pl.lit("50/50").alias("initial conditions"),
            pl.lit("well-mixed").alias("population structure"),
        ),
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/seuyf/download"),
            ),
        ).with_columns(
            pl.lit("de novo").alias("initial conditions"),
            pl.lit("1D demes").alias("population structure"),
        ),
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/ag4ur/download"),
            ),
        ).with_columns(
            pl.lit("de novo").alias("initial conditions"),
            pl.lit("2D demes").alias("population structure"),
        ),
        pl.from_pandas(
            calc_fixprobs_from_traits(
                pd.read_parquet("https://osf.io/edwbu/download"),
            ),
        ).with_columns(
            pl.lit("de novo").alias("initial conditions"),
            pl.lit("well-mixed").alias("population structure"),
        ),
    ],
)

display(dfxs.describe()), display(dfxs.head()), display(dfxs.tail());


## Size Fixation Cliffplot


In [ ]:
for errorbar in "sd", "se", "ci", None:
    with tp.teed(
        sns.relplot,
        data=dfxs.filter(
            pl.col("genotype") == "hypermutator",
        ).with_columns(
            pl.col("fixation probability").alias("fix\nprob"),
            pl.col("population structure").alias("population\nstructure"),
            pl.col("initial conditions").alias("initial\nconditions"),
        ),
        x="available beneficial mutations",
        y="fix\nprob",
        row="population size",
        hue="population\nstructure",
        hue_order=["well-mixed", "1D demes", "2D demes"],
        style="initial\nconditions",
        style_order=["de novo", "50/50"],
        aspect=10,
        errorbar=errorbar,
        # facet_kws=dict(margin_titles=True),
        height=0.8,
        kind="line",
        markers=True,
        palette="Dark2",
        seed=1,
        teeplot_subdir=teeplot_subdir,
    ) as teed:
        teed.set_titles(
            col_template="{{col_var}} = {{col_name:.0f}}",
        )
        teed.set(ylim=(0, 1), xlim=(1, 40))
        sns.move_legend(
            teed,
            "lower center",
            bbox_to_anchor=(0.4, 0.97),
            frameon=False,
            ncol=7,
            title=None,
            columnspacing=0.7,
        )
        for ax in teed.axes.flat:
            ax.axhline(0.5, ls=":", c="gray", lw=1)
            ax.set_yticks([0, 0.5, 1])
            ax.set_yticklabels(["", "0.5", ""])
        teed.tight_layout()
